In [1]:
import zipfile
from glob import glob
from tifffile import imread
#from cellpose import models #in case models are nto downloaded yet

# DISPLAY RESULTS imports
#from cellpose import plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
from scipy.ndimage import zoom
import skimage.io
from PIL import Image
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
import torchvision.transforms as T
import numpy as np
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
import time
import os
import pandas as pd
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import argparse
import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from scipy.ndimage import morphology
sys.path.append(os.path.join(sys.path[0]))  # To find local version of the library

from torchvision.models.detection.rpn import AnchorGenerator
import torch
from skimage.color import rgb2gray, label2rgb
from skimage import io,exposure, feature, filters, io, measure, morphology, restoration, segmentation, transform, util, data, color
from skimage.measure import label, regionprops
from skimage.transform import rescale, resize, downscale_local_mean
import os
import numpy as np
import os
import torch.nn as nn
#load image
from glob import glob
import numpy as np
from skimage.morphology import remove_small_objects,dilation, erosion, ball  
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import matplotlib.pyplot as plt
import skimage
from scipy.ndimage import zoom
import copy
import os
import scipy.ndimage
from skimage import measure, morphology
import gc
from skimage.util import invert
from skimage.morphology import remove_small_objects,dilation, erosion, ball  
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import matplotlib.pyplot as plt
import skimage
from scipy.ndimage import zoom
import copy
import os
import scipy.ndimage
from skimage import measure, morphology
import gc
from skimage.util import invert
import pandas as pd
from vedo import *
import pyvista as pv
from skimage.segmentation import watershed
from skimage.transform import rescale
import torch
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
import torchvision.transforms as T
import numpy as np
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabHead



class Material:
 
  def __init__(self, name, input_rgb_vals, output_val, confidence_threshold=0):
    self.name = name
    self.input_rgb_vals = input_rgb_vals
    self.output_val = output_val
    self.confidence_threshold = confidence_threshold

In [2]:
import torch
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:

model_out="E:\\ANSTO_20960\\models\\"

model_group='devin_cropped/'
current_model_name = 'rhizosheath_ai'
model_number="4"

In [4]:
#root or not root
#Creating a list of materials so we can iterate through it
materials = [
            Material("kapton", [36,36,36], 36 , 0.3),
            Material("root", [73,73,73], 73, 0.4),
            Material("primary", [109,109,109], 109, 0.5),
            Material("mixed", [146,146,146], 146, 0.5),
            Material("root_hair", [182,182,182], 182, 0.05),
            Material("pore", [219,219,219], 219, 0.5),
            Material("organic", [255,255,255], 255, 0.6)]

num_materials =len(materials)
#Decrease scale to decrease VRAM usage; if you run out of VRAM during traing, restartyour runtime and down scale your images
scale=1
#Input model directory
models_directory = model_out
#Input the name you want to use for your group of models
model_group=model_group
current_model_name = current_model_name
model_number=model_number


###############################
# Load FCN
model=torchvision.models.segmentation.fcn_resnet101(pretrained=False)
model.classifier=FCNHead(2048, num_materials)
device = torch.device('cuda')
outputs=[]
model.to(device)
model.load_state_dict(torch.load(models_directory+model_group + current_model_name+model_number+'.pth'), strict=False)
model.train()
mean=3.0020e-15 #This is silly just copy and paste value from model train - this should be saved as a text file ?
std=1

# # Load deep lab v3
# model=torchvision.models.segmentation.fcn_resnet101(pretrained=False)
# model.classifier=FCNHead(2048, num_materials)
# device = torch.device('cuda')
# outputs=[]
# model.to(device)
# model.load_state_dict(torch.load(models_directory+model_group + current_model_name+model_number+'.pth'), strict=False)
# model.train()
# mean=3.0020e-15 #This is silly just copy and paste value from model train - this should be saved as a text file ?
# std=1



In [5]:
#model.eval()

In [6]:
#Part 2 Segment using AI (Segment)
#Loop through folders#######################################
ct_folder_path_name ='E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\'

ct_folder_paths=np.array((glob(ct_folder_path_name+"*\\", recursive = True)))
########
ct_folder_paths = ct_folder_paths[ct_folder_paths != 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_072_W16_S5_P.tif\\']
ct_folder_paths = ct_folder_paths[ct_folder_paths != 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_073_W1_S5_P.tif\\']
ct_folder_paths = ct_folder_paths[ct_folder_paths != 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_074_WT_S10_P.tif\\']
print (ct_folder_paths)
############################

['E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_076_W16_S4_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_077_W1_S4_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_078_WT_S7_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_079_BALD_S1_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_080_W16_S3_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_081_W1_S2_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_082_W1_S1_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_083_BALD_S4_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_084_W16_S1_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_085_W1_S3_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_086_WT_S9_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_087_BALDY_S7_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_CROPPED\\recon_088_W16_S2_P.tif\\'
 'E:\\ANSTO_20960\\rhizo_ai\\XY_SLICES_C

In [7]:
#ct_folder_paths=ct_folder_paths[ct_folder_paths == "E:\\ANSTO_20960\\tiff_stacks_out\\4x_white_beam\\XY\\recon_082_W1_S1_P.tif\\"]

In [8]:
#########################################
output_base = "E:\\ANSTO_20960\\rhizo_ai\\segmented_out\\"
for folder_x in ct_folder_paths:
    #print (folder_x)
    IMAGE_ID = folder_x.replace(ct_folder_path_name , "")
    IMAGE_ID = IMAGE_ID.replace('\\', '')
    print (IMAGE_ID)
    output_directory = output_base+IMAGE_ID+"\\"
    if os.path.isdir(output_directory):
        out=(output_directory)
    else:
        out=os.mkdir(output_directory)
        
    inference_directory= folder_x
    proceeding="lice_"
    following=".tif"
    dir_name = inference_directory
    filenames = os.listdir(dir_name)
    sort_idx = np.argsort([(int(filename.split(proceeding)[1].split(following)[0])) for filename in filenames])
    for i in sort_idx:
        #makes new directory called "(directory name here) + name in red" that your new images go into
        new_dir_name = output_directory
        if not os.path.exists(new_dir_name):
          os.makedirs(new_dir_name)

        for mat in materials:
          new_dir_name_mat= new_dir_name + mat.name
          if not os.path.exists(new_dir_name_mat):
            os.makedirs(new_dir_name_mat)
        filename = filenames[i]

        image = Image.open(dir_name +'/'+ filenames[i])
        image1 = (image)
        image1 = np.array(image1, dtype='ubyte')
        zeros=np.zeros_like(image1)

        w, h = image.size
        #print(image.size)
        #!!!!!!!!!!!!!!!!!!!!Make sure scale matches!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        scale=scale
        newW, newH = int(scale * w), int(scale * h)
        image=image.resize((newW, newH))
        image_resize=image
        image=np.array(image)
        zeros2=np.zeros_like(image)
        new_im=np.zeros((3, newH, newW))
        new_im[0,:,:]=image
        new_im[1,:,:]=image
        new_im[2,:,:]=image
        image=new_im
        image=torch.from_numpy(image)
        image=T.Normalize(mean=[mean, mean , mean], std=[std, std , std])(image)
        image.unsqueeze_(0)
        image = image.to(device=device, dtype=torch.float32)
        tic=time.time()
        with torch.no_grad():
          mask=model(image)['out']
          mask=nn.Sigmoid()(mask)
        toc=time.time()
        #print('time: '+str(toc-tic))

        image_rescaled=zeros2
        combined_image = np.array(image_rescaled, dtype='ubyte') 

        list_of_mat_tables = []
        for i, mat in enumerate(materials):
          mat_mask = mask.cpu().detach().numpy()[0,i,:,:]
          mat_mask[mat_mask >= mat.confidence_threshold] = mat.output_val
          mat_mask[mat_mask < mat.confidence_threshold] = 0
          mat_mask=np.array(mat_mask, dtype='ubyte')
          combined_image = np.add(combined_image, mat_mask[:,:])

          io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0]+'_' + mat.name + "_mask.png",
                    mat_mask, check_contrast=False)
          np_mat = np.array(mat_mask)


        io.imsave(new_dir_name+'/'+filename.split(following)[0]+'.png', combined_image, check_contrast=False)   
        
        



recon_076_W16_S4_P.tif
recon_077_W1_S4_P.tif
recon_078_WT_S7_P.tif
recon_079_BALD_S1_P.tif
recon_080_W16_S3_P.tif
recon_081_W1_S2_P.tif
recon_082_W1_S1_P.tif
recon_083_BALD_S4_P.tif
recon_084_W16_S1_P.tif
recon_085_W1_S3_P.tif
recon_086_WT_S9_P.tif
recon_087_BALDY_S7_P.tif
recon_088_W16_S2_P.tif
recon_089_WT_S2_P.tif
